In [1]:
import pandas as pd
import re

In [2]:
bank_statement_df = pd.read_excel(r"D:\For Auto Recon\BANK STATEMENT REPORT FOR RECONCILE Phase1-20230807T053815Z-001\BANK STATEMENT REPORT FOR RECONCILE Phase1\Nepal Bank 20-10-2020 to 03-06-2021.xlsx",skiprows=1)
soa_statement_df = pd.read_csv(r"D:\For Auto Recon\SOA REPORT FOR RECONCILE Phase1-20230807T053702Z-001\SOA REPORT FOR RECONCILE Phase1\Commission Report (10_20_2020 to 06_03_2021)Nepal Bank Limitted.csv",encoding='latin-1')


In [3]:
bank_statement_df.head()

,CyCode,MainCode,TimeStamp,Desc1,Desc2,Desc3,Unnamed: 6,Amount,TranId,EnteredBy,NostroBranch,amount
0,1.0,24208406906844000001,20/10/2020 10:42:56 AM,9819310353,"NPS1,1931140",100000095626,NaN,500.0,BV5104256,_BSUsr,10.0,500.0
1,1.0,24208406906844000001,21/10/2020 4:04:27 PM,9851047706,"NPS1,1939800",100000099007,NaN,500.0,BV5160427,_BSUsr,2.0,500.0
2,1.0,24208406906844000001,22/10/2020 2:04:45 PM,9851070954,"NPS1,1945533",100000101025,NaN,500.0,BV5140445,_BSUsr,191.0,500.0
3,1.0,24208406906844000001,23/10/2020 4:50:53 PM,9824204012,"NPS1,1952017",100000103041,NaN,1000.0,BV5165053,_BSUsr,211.0,1000.0
4,1.0,24208406906844000001,24/10/2020 6:05:14 PM,9840123770,"NPS1,1956883",100000104397,NaN,500.0,BV5180514,_BSUsr,52.0,500.0


In [20]:
for index, row in bank_statement_df.iterrows():
    if  'LWT' in str(row['Desc3']):
        tran_id = str(row['Desc3']).split()
        id = tran_id[0]
        bank_statement_df.loc[index, 'Transaction Id'] = id
    elif '1000000' in str(row['Desc3']):
        tran_id = str(row['Desc3']).split()
        value = tran_id[0]
        id = re.sub(r'10*', '', value, count=1)
        bank_statement_df.loc[index, 'Transaction Id'] = id
    elif '100000' in str(row['Desc3']):
        tran_id = str(row['Desc3']).split()
        value = tran_id[0]
        id = re.sub(r'10*', '', value, count=1)
        bank_statement_df.loc[index, 'Transaction Id'] = id

In [21]:
bank_statement_df.head()

,CyCode,MainCode,TimeStamp,Desc1,Desc2,Desc3,Unnamed: 6,Amount,TranId,EnteredBy,NostroBranch,amount,Transaction Id
0,1.0,24208406906844000001,20/10/2020 10:42:56 AM,9819310353,"NPS1,1931140",100000095626,NaN,500.0,BV5104256,_BSUsr,10.0,500.0,95626
1,1.0,24208406906844000001,21/10/2020 4:04:27 PM,9851047706,"NPS1,1939800",100000099007,NaN,500.0,BV5160427,_BSUsr,2.0,500.0,99007
2,1.0,24208406906844000001,22/10/2020 2:04:45 PM,9851070954,"NPS1,1945533",100000101025,NaN,500.0,BV5140445,_BSUsr,191.0,500.0,101025
3,1.0,24208406906844000001,23/10/2020 4:50:53 PM,9824204012,"NPS1,1952017",100000103041,NaN,1000.0,BV5165053,_BSUsr,211.0,1000.0,103041
4,1.0,24208406906844000001,24/10/2020 6:05:14 PM,9840123770,"NPS1,1956883",100000104397,NaN,500.0,BV5180514,_BSUsr,52.0,500.0,104397


In [22]:
bank_statement_df['Transaction Id'].count()


10239

In [26]:
bank_statement_df['Date']= pd.to_datetime(bank_statement_df['TimeStamp']).dt.date

0        2020-10-20
1        2020-10-21
2        2020-10-22
3        2020-10-23
4        2020-10-24
            ...    
18143    2021-02-06
18144    2021-05-27
18145    2021-04-06
18146    2021-03-06
18147           NaT
Name: TimeStamp, Length: 18148, dtype: object

In [28]:
pd.to_datetime(soa_statement_df['Date']).dt.date

0        2020-11-20
1        2020-11-20
2        2020-11-20
3        2020-11-20
4        2020-11-21
            ...    
10448    2021-04-26
10449    2021-04-26
10450    2021-04-26
10451    2021-04-26
10452           NaT
Name: Date, Length: 10453, dtype: object

In [30]:
def standard_format(row):
        if row['Amount'] < 0:
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Amount'] > 0:
            mode = 'CR'
            amount = row['Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format, axis=1)    

In [25]:
# bank_statement_df.loc[10530:10560]

In [31]:
bank_statement_df.head()

,CyCode,MainCode,TimeStamp,Desc1,Desc2,Desc3,Unnamed: 6,Amount,TranId,EnteredBy,NostroBranch,amount,Transaction Id,Mode
0,1.0,24208406906844000001,20/10/2020 10:42:56 AM,9819310353,"NPS1,1931140",100000095626,NaN,500.0,BV5104256,_BSUsr,10.0,500.0,95626,CR
1,1.0,24208406906844000001,21/10/2020 4:04:27 PM,9851047706,"NPS1,1939800",100000099007,NaN,500.0,BV5160427,_BSUsr,2.0,500.0,99007,CR
2,1.0,24208406906844000001,22/10/2020 2:04:45 PM,9851070954,"NPS1,1945533",100000101025,NaN,500.0,BV5140445,_BSUsr,191.0,500.0,101025,CR
3,1.0,24208406906844000001,23/10/2020 4:50:53 PM,9824204012,"NPS1,1952017",100000103041,NaN,1000.0,BV5165053,_BSUsr,211.0,1000.0,103041,CR
4,1.0,24208406906844000001,24/10/2020 6:05:14 PM,9840123770,"NPS1,1956883",100000104397,NaN,500.0,BV5180514,_BSUsr,52.0,500.0,104397,CR
